In [ ]:
%cd ~
import matplotlib
matplotlib.use("PDF")
import pyfits,glob,time,scipy, sys
import scipy.interpolate
from scipy.special import erf
from pylab import *
from subprocess import call
from IPython import parallel
from scipy.optimize import fmin_powell, minimize
from scipy.special import airy,jv

topdir='/Users/cslage/Research/LSST/code/GUI/'
#thedir=topdir+'brighter_fatter/20170831_002_spots/'
thedir=topdir+'brighter_fatter/20170609-bf-30um/'
sys.path.append('/Users/cslage/Research/LSST/code/galsim-newgit/GalSim/devel/brighter-fatter/forward_model_varying_i')

import  forward
sys.path.append('/Users/cslage/Learning/code/languages/python/extensions/forward_model_convolved_old')
import  forward_convolved

%cd $thedir


configfile=topdir+'sextractor/default-array_dither.sex'
paramfile=topdir+'sextractor/default-array_dither.param'
maskdir=topdir+'sextractor/masks/'



In [ ]:
def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
    overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
    image=image[:y_overscan_start,:x_overscan_start]
    finalimg=image-matrix(median(overscan,axis=1)).T*np.ones(shape(image)[1])
    return array(finalimg)

def make_reg_from_ldac(cat_ldac_file,text_tag):
    thecat=pyfits.getdata(cat_ldac_file,'LDAC_OBJECTS')
    f = open(cat_ldac_file+'.reg','w')
    for i in range(len(thecat)):
        xcoo,ycoo=thecat['XWIN_IMAGE'][i],thecat['YWIN_IMAGE'][i]
        r=thecat['A_IMAGE'][i]
        thetext=thecat[text_tag][i]
        f.write('circle '+str(xcoo)+' '+str(ycoo)+' '+str(r)+'#text="'+str(thetext)+'"\n')
    f.close()
    
def Area(xl, xh, yl, yh, sigmax, sigmay, Imax):
    # Calculates how much of a 2D Gaussian falls within a rectangular box
    ssigx = sqrt(2) * sigmax
    ssigy = sqrt(2) * sigmay    
    I = (erf(xh/ssigx)-erf(xl/ssigx))*(erf(yh/ssigy)-erf(yl/ssigy))
    return Imax * I / 4.0

In [ ]:
# This definition runs sextractor in parallel and can be given to IPython's parallel map function along with
#   the file list
def ovsub_runsex_makereg(fitsfilename):
    import pyfits
    import numpy as np
    from subprocess import call
    topdir='/Users/cslage/Research/LSST/code/GUI/'
    thedir=topdir+'profiles'
    %cd $thedir

    configfile=topdir+'sextractor/default-array_dither.sex'
    paramfile=topdir+'sextractor/default-array_dither.param'
    maskdir=topdir+'sextractor/masks/'

    def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
        overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
        image=image[:y_overscan_start,:x_overscan_start]
        finalimg=image-np.matrix(np.median(overscan,axis=1)).T*np.ones(np.shape(image)[1])
        return finalimg

    def make_reg_from_ldac(cat_ldac_file,text_tag):
        thecat=pyfits.getdata(cat_ldac_file,'LDAC_OBJECTS')
        f = open(cat_ldac_file+'.reg','w')
        for i in range(len(thecat)):
            xcoo,ycoo=thecat['XWIN_IMAGE'][i],thecat['YWIN_IMAGE'][i]
            r=thecat['A_IMAGE'][i]
            thetext=thecat[text_tag][i]
            f.write('circle '+str(xcoo)+' '+str(ycoo)+' '+str(r)+'#text="'+str(thetext)+'"\n')
        f.close()
    for i in range(1,17):
        extname=pyfits.getheader(fitsfilename,i)['EXTNAME']
        img=pyfits.getdata(fitsfilename,extname)
        overscansubimg=remove_overscan_xy(img,509,542,2000,2022)   # cut off the overscan
        outname=fitsfilename[:-5]+extname+'.fits'
        pyfits.writeto(outname,overscansubimg,clobber=True)
        test=call(["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat'])
        make_reg_from_ldac(outname+'.cat','NUMBER')

In [ ]:
zfilelist=sort(glob.glob(thedir+'ITL-3800C-002_spot_spot_810_20170609??????.fits'))
print len(zfilelist)

In [ ]:
for fitsfilename in zfilelist: 
    tfile1=time.time() 
    for i in [13]:
        tstart=time.time() 
        extname=pyfits.getheader(fitsfilename,i)['EXTNAME'] 
        img=pyfits.getdata(fitsfilename,extname) 
        overscansubimg=remove_overscan_xy(img,509,542,2000,2022) 
        # cut off the overscan 
        outname=fitsfilename[:-5]+extname+'.fits' 
        pyfits.writeto(outname,overscansubimg,clobber=True) 
        test=call(["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat']) 
        make_reg_from_ldac(outname+'.cat','NUMBER') 
        tend=time.time() 
        print extname+" time: "+str(tend-tstart)[:4] 
        print "Time taken for file "+str(fitsfilename[-26:-23])+": "+str(time.time()-tfile1)

#### Using the sextractor catalogs produced above to make a map of the sextractor measurement named below

In [ ]:
class Array2dSet:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny,nstamps):
        # This packages up a set of nstamps postage stamp images,
        # each image of which is nx * ny pixels
        self.nx=nx
        self.ny=ny
        self.nstamps=nstamps

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=zeros([nx,ny,nstamps])
        self.xoffset=zeros([nstamps])
        self.yoffset=zeros([nstamps])
        self.imax=zeros([nstamps])


def BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny, minsize, maxsize):
    stampxmin = -(int(nx/2)+0.5)
    stampxmax = -stampxmin
    stampymin = -(int(ny/2)+0.5)
    stampymax = -stampymin
    xcoomin = 50
    xcoomax = 450
    ycoomin = 1400
    ycoomax = 1900
    spotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,numspots)
    hdr=pyfits.getheader(fitsfilename,segmentnumber)
    extname = hdr['EXTNAME']
    img=pyfits.getdata(fitsfilename,extname) 
    overscansubimg=remove_overscan_xy(img,509,542,2000,2022) 
    catname=fitsfilename[:-5]+extname+'.fits.cat.reg' 
    catfile = open(catname,'r')
    catlines = catfile.readlines()
    catfile.close()
    n=0
    for line in catlines:
        try:
            size = float(line.split()[3].split('#')[0])
            if size < minsize or size > maxsize:
                continue
            xcoord = float(line.split()[1])
            ycoord = float(line.split()[2])
            if xcoord < xcoomin or xcoord > xcoomax or ycoord < ycoomin or ycoord > ycoomax:
                continue
            xint = int(xcoord-0.5)
            yint = int(ycoord-0.5)
            xmin = xint - int(nx/2)
            xmax = xint + int(nx/2) + 1
            ymin = yint - int(ny/2)
            ymax = yint + int(ny/2) + 1
            stamp = overscansubimg[ymin:ymax, xmin:xmax]
           
            xsum = 0.0
            ysum = 0.0
            datasum = 0.0
             
            for i in range(nx):
                for j in range(ny):
                    spotlist.data[i,j,n] = float(stamp[j,i])                    
                    ysum += spotlist.y[j] * spotlist.data[i,j,n]
                    xsum += spotlist.x[i] * spotlist.data[i,j,n]
                    datasum += spotlist.data[i,j,n]
            xoff = xsum / datasum
            yoff = ysum / datasum
            spotlist.xoffset[n] = xoff#xcoord - xint - 1.0
            spotlist.yoffset[n] = yoff#ycoord - yint - 1.0     
                    
            n += 1
            if n == numspots:
                return spotlist
        except:
            continue
    # Reaching this point means we found less spots than requested.
    newspotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,n)
    newspotlist.xoffset = spotlist.xoffset[0:n]
    newspotlist.yoffset = spotlist.yoffset[0:n]
    newspotlist.data = spotlist.data[:,:,0:n]
    del spotlist
    return newspotlist

def FOM(params):
    [sigmax, sigmay] = params
    result = forward.forward(spotlist,sigmax,sigmay)
    return result

def PyFOM(params):
    fom = 0.0
    [Imax, sigmax, sigmay] = params
    area=zeros([spotlist.nx,spotlist.ny])
   
    for spot in range(spotlist.nstamps):
        for ii in range(spotlist.nx):
            for jj in range(spotlist.ny):
                xl = spotlist.x[ii] - spotlist.xoffset[spot] - 0.5
                xh = xl + 1.0
                yl = spotlist.y[jj] - spotlist.yoffset[spot] - 0.5
                yh = yl + 1.0
                #if spot == 78 and ii == 4 and jj == 4 or spot==0:
                    #print "ii = %d, jj = %d, img = %.4f"%(ii,jj,spotlist.data[ii,jj,spot])
                #print "ii = %d, jj = %d,xl = %.2f, xh = %.2f, yl = %.2f, yh = %.2f"%(ii,jj,xl,xh,yl,yh)
                area[ii,jj] = Area(xl, xh, yl, yh, sigmax, sigmay, Imax)
                fom += square(area[ii,jj]-spotlist.data[ii,jj,spot])
    #print "Imax = %.1f, sigmax = %.2f, sigmay = %.2f, fom = %.1f"%(Imax, sigmax, sigmay, fom)
    return fom


In [ ]:
nx = 11
ny = 11
numspots =400
segmentnumber = 13

fitsfilename = zfilelist[0]
print fitsfilename
    
spotlist = BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny, 0.7, 1.4)

print "nstamps = %d"%spotlist.nstamps
starttime = time.time()
param0 = [1.00, 1.00]

args = ()#(spotlist)
Result = fmin_powell(FOM, param0, args)
elapsed = time.time() - starttime
print Result, "Elapsed Time = %f"%elapsed
testn = 0
spot = 228
sigmax = abs(Result[0])
sigmay = abs(Result[1])
ADU_correction = Area(-0.5,0.5,-0.5,0.5,sigmax, sigmay,1.0)
Imax = spotlist.imax[spot] 
figure(figsize=(12,5))
suptitle=("Spot # %d"%(spot))
area=zeros([nx,ny])
for ii in range(nx):
    for jj in range(ny):
        xl = ii - int(nx/2) - spotlist.xoffset[spot] - 0.5
        xh = xl + 1.0
        yl = jj - int(ny/2) - spotlist.yoffset[spot] - 0.5
        yh = yl + 1.0
        area[ii,jj] = Area(xl, xh, yl, yh, sigmax, sigmay, Imax) 

subplots_adjust(wspace = 2.0)
subplot(1,3,1)
imshow(spotlist.data[:,:,spot] ,interpolation="None")
subplot(1,3,2)
plot(spotlist.data[int(nx/2),:,spot] , lw = 2, label="Data")
plot(area[int(nx/2),:], lw = 2, label="Model")
xlabel("X (Pixels)")
ylabel("Signal(ADU)")
xticks([0,2,4,6,8,10])
subplot(1,3,3)
plot(spotlist.data[:,int(ny/2),spot], lw = 2, label="Data")
plot(area[:,int(ny/2)], lw = 2, label="Model")
xlabel("Y (Pixels)")
ylabel("Signal(ADU)")
xticks([0,2,4,6,8,10])
legend(loc = (-6.5,0.8))
savefig("Typical_Fit_20170609_08Nov17_%s.pdf"%(testn))
show()
close()


In [ ]:

class Array2d:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny):
        self.nx=nx
        self.ny=ny

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=zeros([nx,ny])

def CGIntegrand(y, xh, yh, sigmax, sigmay, c):
    ssx = sigmax * sqrt(2.0)
    ssy = sigmay * sqrt(2.0)
    yc = sqrt(c*c - y*y)
    yp = yc + xh
    ym = yc - xh
    f = yp * erf(yp / ssx) - ym * erf(ym / ssx)
    f += ssx * (exp(-yp * yp / (ssx * ssx)) - exp(-ym * ym / (ssx * ssx))) / sqrt(pi)
    f *= erf((y+yh) / ssy)
    return f

def ICGStep(xh, yh, sigmax, sigmay, c):
    ny = 1000
    dy = 2 * c / (ny - 1)
    sum = 0.0
    y = -c + dy / 2.0
    for i in range(ny-1):
        sum += dy * CGIntegrand(y,xh,yh,sigmax,sigmay,c)
        y += dy
    return sum

"""
def ICGFull(xl, xh, yl, yh, sigmax, sigmay, c):
    sum = ICGStep(xh,yh,sigmax,sigmay,c) + ICGStep(xl,yl,sigmax,sigmay,c)
    sum -= ICGStep(xh,yl,sigmax,sigmay,c) + ICGStep(xl,yh,sigmax,sigmay,c)
    return sum
"""
from scipy.integrate import quad

def ICGFull(xl, xh, yl, yh, sigmax, sigmay, c):
    sum = quad(CGIntegrand, -c, c, args=(xh,yh,sigmax,sigmay,c))[0]
    sum += quad(CGIntegrand, -c, c, args=(xl,yl,sigmax,sigmay,c))[0]
    sum -= quad(CGIntegrand, -c, c, args=(xh,yl,sigmax,sigmay,c))[0]
    sum -= quad(CGIntegrand, -c, c, args=(xl,yh,sigmax,sigmay,c))[0]
    return sum

        
def CGFOM(params):
    [sigmax, sigmay] = params
    #[sigmax, sigmay, c] = params
    c=1.63
    print "Calling forward_convolved."
    sys.stdout.flush();
    result = forward_convolved.forward_convolved(spotlist,sigmax,sigmay,c)
    return result


nx = 11
ny = 11
numspots =400
segmentnumber = 13

fitsfilename = zfilelist[0]
print fitsfilename
sys.stdout.flush();
    
spotlist = BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny, 0.7, 1.4)

print "nstamps = %d"%spotlist.nstamps
starttime = time.time()
#param0 = [1.00, 1.00, 0.7]
param0 = [1.00, 1.00]

args = ()#(spotlist)
Result = fmin_powell(CGFOM, param0, args)

elapsed = time.time() - starttime
print Result, "Elapsed Time = %f"%elapsed


In [ ]:
testn = 0
spot = 228
sigmax = abs(Result[0])
sigmay = abs(Result[1])
c = 1.63#Result[2]
norm = ICGFull(-10.0,10.0,-10.0,10.0,sigmax,sigmay,c)
ADU_correction = ICGFull(-0.5,0.5,-0.5,0.5,sigmax, sigmay,c) / norm
Imax = spotlist.imax[spot] 
figure(figsize=(12,5))
suptitle=("Spot # %d"%(spot))
area=zeros([nx,ny])
for ii in range(nx):
    for jj in range(ny):
        xl = ii - int(nx/2) - spotlist.xoffset[spot] - 0.5
        xh = xl + 1.0
        yl = jj - int(ny/2) - spotlist.yoffset[spot] - 0.5
        yh = yl + 1.0
        area[ii,jj] = Imax * ICGFull(xl, xh, yl, yh, sigmax, sigmay, c) / norm

subplots_adjust(wspace = 2.0)
subplot(1,3,1)
imshow(spotlist.data[:,:,spot] ,interpolation="None")
subplot(1,3,2)
plot(spotlist.data[int(nx/2),:,spot] , lw = 2, label="Data")
plot(area[int(nx/2),:], lw = 2, label="Model")
xlabel("X (Pixels)")
ylabel("Signal(ADU)")
xticks([0,2,4,6,8])
ylim(0,45000)
subplot(1,3,3)
plot(spotlist.data[:,int(ny/2),spot], lw = 2, label="Data")
plot(area[:,int(ny/2)], lw = 2, label="Model")
xlabel("Y (Pixels)")
ylabel("Signal(ADU)")
xticks([0,2,4,6,8])
ylim(0,45000)
legend(loc = (-6.5,0.8))
savefig("Typical_Fit_Convolved_08Nov17_%s.pdf"%(testn))
show()
close()


In [ ]:

class Array2d:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny):
        self.nx=nx
        self.ny=ny

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=zeros([nx,ny])

def CGIntegrand(x, y, sigmax, sigmay, c):
    ssigx = sigmax * sqrt(2.0)
    ssigy = sigmay * sqrt(2.0)
    sr = sqrt((x/ssigx)**2 + (y/ssigy)**2)
    return (erf(sr+c) - erf(sr-c)) / (2.0 * erf(c))

def ICG(xl, xh, yl, yh, sigmax, sigmay, c):
    # This is normalized to have an Integral of 1.0
    NormFactor = 2.0*pi*sigmax*sigmay*(c*c+0.5) + 2.0*sigmax*sigmay*sqrt(pi)*c/erf(c)*exp(-c*c)
    nx = 100
    ny = 100
    data = Array2d(xl,xh,nx,yl,yh,ny)
    sum = 0.0
    for i in range(nx):
        for j in range(ny):
            data.data[i,j] = CGIntegrand(data.x[i],data.y[j],sigmax=sigmax, sigmay=sigmay, c=c)
            sum += data.data[i,j] * data.dx * data.dy
    return sum / NormFactor

        
def CGFOM(params):
    #[sigmax, sigmay] = params
    [sigmax, sigmay, c] = params
    #c=1.63
    print "Calling forward_convolved."
    sys.stdout.flush();
    result = forward_convolved.forward_convolved(spotlist,sigmax,sigmay,c)
    return result


nx = 11
ny = 11
numspots =4
segmentnumber = 13

fitsfilename = zfilelist[0]
print fitsfilename
sys.stdout.flush();
    
spotlist = BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny, 0.7, 1.4)

print "nstamps = %d"%spotlist.nstamps
starttime = time.time()
param0 = [1.00, 1.00, 0.7]
#param0 = [1.00, 1.00]

args = ()#(spotlist)
Result = fmin_powell(CGFOM, param0, args)

elapsed = time.time() - starttime
print Result, "Elapsed Time = %f"%elapsed


In [ ]:
testn = 0
spot = 2
sigmax = abs(Result[0])
sigmay = abs(Result[1])
c = Result[2]
ADU_correction = ICG(-0.5,0.5,-0.5,0.5,sigmax, sigmay,c)
Imax = spotlist.imax[spot] 
figure(figsize=(12,5))
suptitle=("Spot # %d"%(spot))
area=zeros([nx,ny])
for ii in range(nx):
    for jj in range(ny):
        xl = ii - int(nx/2) - spotlist.xoffset[spot] - 0.5
        xh = xl + 1.0
        yl = jj - int(ny/2) - spotlist.yoffset[spot] - 0.5
        yh = yl + 1.0
        area[ii,jj] = Imax * ICG(xl, xh, yl, yh, sigmax, sigmay, c)

subplots_adjust(wspace = 2.0)
subplot(1,3,1)
imshow(spotlist.data[:,:,spot] ,interpolation="None")
subplot(1,3,2)
plot(spotlist.data[int(nx/2),:,spot] , lw = 2, label="Data")
plot(area[int(nx/2),:], lw = 2, label="Model")
xlabel("X (Pixels)")
ylabel("Signal(ADU)")
xticks([0,2,4,6,8])
ylim(0,45000)
subplot(1,3,3)
plot(spotlist.data[:,int(ny/2),spot], lw = 2, label="Data")
plot(area[:,int(ny/2)], lw = 2, label="Model")
xlabel("Y (Pixels)")
ylabel("Signal(ADU)")
xticks([0,2,4,6,8])
ylim(0,45000)
legend(loc = (-6.5,0.8))
savefig("Typical_Fit_Convolved_Old_10Nov17_%s.pdf"%(testn))
show()
close()


In [ ]:
def CGIntegrandP(y, xh, yh, sigmax, sigmay, c):
    ssx = sigmax * sqrt(2.0)
    ssy = sigmay * sqrt(2.0)
    yc = sqrt(c*c - y*y)
    yp = yc + xh
    f = yp * erf(yp / ssx)
    f *= erf((y+yh) / ssy)
    return f

def CGIntegrandM(y, xh, yh, sigmax, sigmay, c):
    ssx = sigmax * sqrt(2.0)
    ssy = sigmay * sqrt(2.0)
    yc = sqrt(c*c - y*y)
    ym = yc - xh
    f = ym * erf(ym / ssx)
    f *= erf((y+yh) / ssy)
    return f

def CGIntegrand3(y, xh, yh, sigmax, sigmay, c):
    ssx = sigmax * sqrt(2.0)
    ssy = sigmay * sqrt(2.0)
    yc = sqrt(c*c - y*y)
    yp = yc + xh
    ym = yc - xh
    f = ssx * (exp(-yp * yp / (ssx * ssx)) - exp(-ym * ym / (ssx * ssx))) / sqrt(pi)
    f *= erf((y+yh) / ssy)
    return f





close()
sigmax = 0.7
sigmay = 0.6
xh = 1.23
yh = 0.78
x = linspace(-c,c,500)
y = CGIntegrand(x, xh, yh, sigmax, sigmay, c)
yp = CGIntegrandP(x, xh, yh, sigmax, sigmay, c)
ym = -CGIntegrandM(x, xh, yh, sigmax, sigmay, c)
y3 = -CGIntegrand3(x, xh, yh, sigmax, sigmay, c)
plot(x,y,color='blue')
plot(x,yp, color='green', ls='--')
plot(x,ym, color='red', ls='--')
plot(x,y3, color='black', ls='--')
show()

In [ ]:
from scipy.integrate import quad
close()
sigmax = 0.7
sigmay = 0.6
c = 1.63
xl = 0.23
yl = -0.22
xh = 1.23
yh = 0.78

I = quad(CGIntegrand, -c, c, args=(xh,yh,sigmax,sigmay,c))
I2 = ICGStep(xh, yh, sigmax, sigmay, c)
print I, I2

def ICGFullInt(xl, xh, yl, yh, sigmax, sigmay, c):
    sum = quad(CGIntegrand, -c, c, args=(xh,yh,sigmax,sigmay,c))[0]
    sum += quad(CGIntegrand, -c, c, args=(xl,yl,sigmax,sigmay,c))[0]
    sum -= quad(CGIntegrand, -c, c, args=(xh,yl,sigmax,sigmay,c))[0]
    sum -= quad(CGIntegrand, -c, c, args=(xl,yh,sigmax,sigmay,c))[0]
    return sum

I = ICGFullInt(xl, xh, yl, yh, sigmax, sigmay, c)
I2 = ICGFull(xl, xh, yl, yh, sigmax, sigmay, c)
print I, I2


